In [0]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np 
import pandas as pd 
import os

# to make this notebook's output stable across runs
# any number will do, as long as it is used consistently
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [0]:
# transform and standardize data
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin

## **STEP ONE: Get the data**
Virginia Department of Education releases statistics and information concerning dropout rates every year. We are using the last five years of released records (2013 to 2017.) Keep in mind that we cannot access related education data for years 2018 and 2019. 

In [0]:
# Load all datasets
dropout2017 = pd.read_csv("http://www.doe.virginia.gov/statistics_reports/research_data/data_files/dropouts/dropouts-2017-results.csv")
dropout2016 = pd.read_csv("http://www.doe.virginia.gov/statistics_reports/research_data/data_files/dropouts/dropouts-2016-results.csv")
dropout2015 = pd.read_csv("http://www.doe.virginia.gov/statistics_reports/research_data/data_files/dropouts/dropouts-2015-results.csv")
dropout2014 = pd.read_csv("http://www.doe.virginia.gov/statistics_reports/research_data/data_files/dropouts/dropouts-2014-results.csv")
dropout2013 = pd.read_csv("http://www.doe.virginia.gov/statistics_reports/research_data/data_files/dropouts/dropouts-2013-results.csv")

dropout = dropout2017.append([dropout2016, dropout2015, dropout2014, dropout2013], ignore_index=True)
dropout.head()

,SCHOOL_YEAR,LEVEL_CODE,DIV_NUM,DIV_NAME,SCH_NUM,SCH_NAME,FEDERAL_RACE_CODE,GENDER,GRADE_CODE,DISABILITY_FLAG,DISADVANTAGED_FLAG,LEP_FLAG,DROPOUT_CNT
0,2017-2018,STATE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7541
1,2017-2018,STATE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,5506
2,2017-2018,STATE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,2035
3,2017-2018,STATE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,3942
4,2017-2018,STATE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,2914


In [0]:
dropout.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100046 entries, 0 to 100045
Data columns (total 13 columns):
SCHOOL_YEAR           100046 non-null object
LEVEL_CODE            100046 non-null object
DIV_NUM               90036 non-null float64
DIV_NAME              90036 non-null object
SCH_NUM               42190 non-null float64
SCH_NAME              42190 non-null object
FEDERAL_RACE_CODE     48136 non-null float64
GENDER                47966 non-null object
GRADE_CODE            46429 non-null float64
DISABILITY_FLAG       48472 non-null object
DISADVANTAGED_FLAG    48139 non-null object
LEP_FLAG              50291 non-null object
DROPOUT_CNT           100046 non-null int64
dtypes: float64(4), int64(1), object(8)
memory usage: 9.9+ MB


## **STEP TWO: Data Wrangling**
Machine Learning models are generally very sensitive to unstandardized data; here, we are using both sklearn transformers and a custom transformer. 
### Custom Transformer `replaceVals`
While we can use the standard sklearn transformers for imputing values and encoding, we also want to encode the flags. This cannot easily be done with the standard sklearn transformers. 

`replaceVals` is custom transformer that encodes the "Y, N, M, F" values on a -1, 0, 1 scale. We are hoping that the symmetry would keep the weight around 0. 



In [0]:
class replaceVals(BaseEstimator, TransformerMixin):
    def __init__(self):
        return
    def fit(self, X):
        return self 
    def transform(self, X):
      X[X == 'Y'] = 1
      X[X == 'N'] = -1
      X[X == 'M'] = 1
      X[X == 'F'] = -1
      
      return X

### Creating the Pipeline
In order to properly transform the data, we need to specify where to replace missing values, where to transform, etc. We also leave out columns (which drops them after standardization.) 

In [0]:
# These are the columns to fill missing values
fill_cols = ['DIV_NUM', 'SCH_NUM', 'FEDERAL_RACE_CODE', 'GENDER',
       'GRADE_CODE', 'DISABILITY_FLAG', 'DISADVANTAGED_FLAG', 'LEP_FLAG', 'DROPOUT_CNT']

# We will only densely one hot encode Level Code
one_hot_cols = ["SCHOOL_YEAR", "LEVEL_CODE"]

In [0]:
# Replacement pipeline Pipeline
replacing_pipeline = Pipeline([           
    ('imputer', SimpleImputer(missing_values=np.nan, strategy="constant", fill_value=0)),
    ("replacements", replaceVals()),
])

# Combined Pipeline
full_pipeline = ColumnTransformer([
    ("cat", OneHotEncoder(), one_hot_cols),
    ("num", replacing_pipeline, fill_cols),
])

### Transforming Data
After creating the pipeline, we use `.fit_transform()` to pass our data through the pipeline. 

Here, we also split the data into training and testing data sets. 

In [0]:
prepped_data = full_pipeline.fit_transform(dropout)
prepped_data = pd.DataFrame(prepped_data)
prepped_data.to_csv(path_or_buf="prepped_data.csv")

In [0]:
prepped_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,7541
1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,-1,5506
2,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,2035
3,0,0,0,0,1,0,0,1,0,0,0,0,0,0,-1,0,3942
4,0,0,0,0,1,0,0,1,0,0,0,0,0,0,-1,-1,2914


In [0]:
"""
0, 1, 2, 3 - school year
4, 5, 6, 7 - level code
8 - div num
9 - sch num
10 - federal race code
11 - gender
12 - grade code
13 - disability flag
14 - disadvantaged flag
15 - lep flag
16 - dropout count
"""